### 导入包及数据部分

In [ ]:
import collections
from collections import defaultdict
from os.path import join

import matplotlib.pyplot as plt
import numpy as np  # 导入numpy，用于科学计算，如，矩阵运算
from sklearn.neighbors import KNeighborsClassifier  # 包装好的knn算法


def file2matrix(filename):
    fr = open(filename)
    number_of_lines = len(
        fr.readlines()
    )  # get the number of lines in the file
    # prepare matrix to return the number of features
    return_mat = np.zeros((number_of_lines, 5))
    class_label_vector = []  # prepare labels return
    fr = open(filename)
    for index, line in enumerate(fr.readlines()):
        line = line.strip()
        list_from_line = line.split("\t")
        return_mat[index, :] = list_from_line[0:5]  # chose features
        class_label_vector.append(float(list_from_line[-1]))
        # classLabelVector.append(float(0))
    return return_mat, class_label_vector


### 生成KNN分类器

In [ ]:
data_X, data_y = file2matrix(join("data", "sample.dat"))
kNN_classifier = KNeighborsClassifier(n_neighbors=10)  # knn中k值
kNN_classifier.fit(data_X, data_y)  # fit 拟合


In [ ]:
def ModulationClassTest(SNR, method, color="#000"):
    accuracy = defaultdict(list)
    # 分别代表 BPSK, QPSK, 16QAM, 64QAM
    numbers = [
        defaultdict(list),
        defaultdict(list),
        defaultdict(list),
        defaultdict(list),
    ]
    labels = ["BPSK", "QPSK", "16QAM", "64QAM"]
    for snr in SNR:
        testDataMat, _ = file2matrix(
            join("data", "test" + labels[method] + "-" + str(snr) + ".dat")
        )
        numTestVecs = testDataMat.shape[0]
        for i in range(4):
            numbers[i][snr] = 0.0
        for i in range(numTestVecs):

            X_predict = testDataMat[i, :].reshape(1, -1)
            y_predict = kNN_classifier.predict(X_predict)

            if y_predict == 2:
                numbers[0][snr] += 1.0
            elif y_predict == 4:
                numbers[1][snr] += 1.0
            elif y_predict == 16:
                numbers[2][snr] += 1.0
            elif y_predict == 64:
                numbers[3][snr] += 1.0

        accuracy[snr] = numbers[method][snr] / numTestVecs
        print("the total correct rate on %d dB SNR is:" % snr, (accuracy[snr]))
        accuracy = collections.OrderedDict(sorted(accuracy.items()))
        for i in range(4):
            print(
                ("正确" if i == method else "") + "判断为",
                labels[i] + ":",
                numbers[i][snr],
            )

    print(accuracy)
    plt.figure(figsize=(8, 5), dpi=100)
    x = SNR
    y = list(accuracy.values())
    plt.plot(x, y, marker="o", linewidth=2.0, linestyle="dashed", color=color)
    plt.axis([0, 10, 0, 1])
    plt.xticks(np.arange(min(x), max(x) + 1, 5.0))
    plt.yticks(np.arange(0, 1, 0.10))

    plt.title("SNR vs Accuracy - " + labels[method], fontsize=16)
    plt.xlabel("SNR (dB)", fontsize=14)
    plt.ylabel("Test accuracy", fontsize=14)
    plt.grid()

    plt.show()


### BPSK ModulationClassTest

In [ ]:
SNR = [2 * x for x in range(-2, 6)]
ModulationClassTest(SNR, 0, "royalblue")

### QPSK ModulationClassTest

In [ ]:
SNR = [2 * x for x in range(-2, 6)]
ModulationClassTest(SNR, 1, "#800080")

### 16QAM ModulationClassTest

In [ ]:
SNR = [5 * x for x in range(0, 9)]
ModulationClassTest(SNR, 2, "#228B22")